# Question Answering su pagine web

In [ ]:
import bs4  # pip install bs4
from langchain_community.document_loaders import WebBaseLoader
import os

bs4_strainer = bs4.SoupStrainer()
# Inizializza il loader per scaricare e processare le pagine web dei corsi Develhope.
# Il loader utilizza BeautifulSoup (bs4) per il parsing HTML e scarica i contenuti dagli URL specificati.
loader = WebBaseLoader(
    web_paths=(
        "https://www.develhope.co/corsi/ai-powered-developer",
        "https://www.develhope.co/corsi/data-science",
        "https://www.develhope.co/corsi/back-end-java-developer",
        "https://www.develhope.co/corsi/full-stack-web-developer",
        "https://www.develhope.co/corsi/front-end-web-developer",
        "https://www.develhope.co/corsi/digital-marketing-ai-specialist",
        "https://www.develhope.co/corsi/ai-data-analyst",
        "https://www.develhope.co/corsi/ux-ui",
    ),
    # Il parametro bs_kwargs permette di passare argomenti aggiuntivi al costruttore di BeautifulSoup.
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [ ]:
docs

In [ ]:
len(docs[0].page_content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=400, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
len(all_splits[42].page_content)

In [ ]:
all_splits[0].metadata

In [ ]:
all_splits[0].page_content

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 8}) # default 4

In [ ]:
retrieved_docs = retriever.invoke("Ho già competenze di programmazione ma voglio fare qualcosa di avanzato e moderno")

In [ ]:
for doc in retrieved_docs:
    print(doc.metadata['title'])
    print(doc.page_content, end="\n\n---\n\n")

In [ ]:
retrieved_docs = retriever.invoke("vorrei diventare specializzarmi nel campo delle AI e delle Agentic Applications")
for doc in retrieved_docs:
    print(doc.metadata['title'])
    print(doc.page_content, end="\n\n---\n\n")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=os.getenv("openai_api_key"))

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

prompt

In [ ]:
example_messages = prompt.invoke(

    {"context": "contesto",
     "question": "domanda"}
     
).to_messages()

example_messages

In [ ]:
print(example_messages[0].content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Creo una chain per elaborare il contesto
rag_chain = (
    
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}

    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("mi piace l'intelligenza artificiale ma non saprei da dove iniziare a studiare, consigliami un corso")

In [ ]:
# Importa la classe RunnableParallel per eseguire più operazioni in parallelo
from langchain_core.runnables import RunnableParallel

# Definisce una catena (chain) che prende dei documenti già selezionati ("context"),
# li formatta come testo, li passa al prompt, poi al modello LLM, e infine estrae solo la risposta testuale
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Definisce una catena parallela che:
# - recupera i documenti rilevanti tramite il retriever
# - passa la domanda così com'è
# Poi, aggiunge un campo "answer" che esegue la catena rag_chain_from_docs sui risultati ottenuti
rag_chain_with_source = RunnableParallel(
    {
        "context": retriever,  # recupera i documenti simili alla domanda
        "question": RunnablePassthrough()  # passa la domanda senza modificarla
    }
).assign(answer=rag_chain_from_docs)  # genera la risposta usando i documenti recuperati

In [ ]:
risposta = rag_chain_with_source.invoke("come potrei approfondire tematiche di Deep Learning?")
risposta

In [24]:
rag_chain_with_source = RunnableParallel(
    
    {"context": retriever,
     "question": RunnablePassthrough()}

) | rag_chain_from_docs

In [25]:
risposta = rag_chain_with_source.invoke("Non mi piace la matematica e vorrei qualcosa di più visuale. Che corso potrei fare?")

risposta

"Potresti considerare di iscriverti al corso di UX/UI Designer, che è un percorso 100% remoto e parte da zero. Questo corso ti insegnerà a sviluppare strategie di design incentrate sull'utente e sui dati, ed è più visivo rispetto alla matematica. In alternativa, il corso di sviluppo web offre una combinazione di aspetti pratici e creativi che potrebbero risultarti interessanti."